This script creates a dataset with time series from a list of cdf names, a start date, an end date and a frequency. Python calls use the MetaTrader5 software, which should be running and conected to an account.

In [ ]:
import MetaTrader5 as mt5
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
from datetime import datetime
import time
import calendar as cal
import os

In [2]:
if not mt5.initialize(): 
    print("initialize() failed") 
    mt5.shutdown() 

In [3]:
print(mt5.terminal_info())

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2561, maxbars=100000, codepage=1252, ping_last=220349, community_balance=0.0, retransmission=0.0, company='Admiral Markets Group AS', name='Admiral Markets MT5', language='Portuguese (Brazil)', path='C:\\Program Files\\Admiral Markets MT5', data_path='C:\\Users\\marceloalleo\\AppData\\Roaming\\MetaQuotes\\Terminal\\24F345EB9F291441AFE537834F9D8A19', commondata_path='C:\\Users\\marceloalleo\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')


In [180]:
def get_time_series (start_time, end_time, frequency, names_list):
    start_time_tuple = time.mktime(start_time.timetuple())
    end_time_tuple = time.mktime(end_time.timetuple())
    end_time_tuple = int(end_time_tuple/900)*900
    names_list.insert(0, 'Date Time')
    dataset = pd.DataFrame(columns = names_list)
    names_list.pop(0)
    cur_time = start_time
    cur_time_tuple = start_time_tuple
    week = 1
    while end_time_tuple - cur_time_tuple >= 0:
        cur_time_string = str(time.ctime(cur_time_tuple))
        #Skip weekends
        if cur_time_string[11:13] == '21' and 'Fri' in cur_time_string[0:4] and cur_time_string[14:16] == '15'and \
        end_time_tuple - cur_time_tuple > 171000:
            print(week)
            cur_time_tuple += 171900
            week += 1
        t = datetime.fromtimestamp(cur_time_tuple).strftime('%m/%d/%Y %H:%M')
        tick_list = [t]
        flag = 0
        for name in names_list:
            tick = mt5.copy_ticks_from(name, datetime.fromtimestamp(cur_time_tuple), 1, mt5.COPY_TICKS_ALL)[0][1]
            tick_list.append(tick)
        dataset.loc[len(dataset)] = tick_list
        cur_time_tuple += frequency
    return dataset

In [181]:
names_list = ["SILVER", "GOLD", "EURAUD", "[DJI30]", "[SP500]", "[DAX30]", "BRENT", "#SVXY", 
              "#China50_Q0", "#USTNote_U0", "#Bund_U0"]
dataset = get_time_series(datetime(2020,1,1,21,0), datetime(2020,8,21,20,0), 900, names_list) 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [186]:
dataset

,Date Time,SILVER,GOLD,EURAUD,[DJI30],[SP500],[DAX30],BRENT,#SVXY,#China50_Q0,#USTNote_U0,#Bund_U0
0,01/01/2020 21:00,17.889,1518.60,1.59917,28597.50,3237.73,13204.95,67.46,65.33,15349.0,138.74,173.83
1,01/01/2020 21:15,17.889,1518.60,1.59917,28597.50,3237.73,13204.95,67.46,65.33,15349.0,138.74,173.83
2,01/01/2020 21:30,17.889,1518.60,1.59917,28597.50,3237.73,13204.95,67.46,65.33,15349.0,138.74,173.83
3,01/01/2020 21:45,17.889,1518.60,1.59917,28597.50,3237.73,13204.95,67.46,65.33,15349.0,138.74,173.83
4,01/01/2020 22:00,17.889,1518.60,1.59917,28597.50,3237.73,13204.95,67.46,65.33,15349.0,138.74,173.83
...,...,...,...,...,...,...,...,...,...,...,...,...
16057,08/21/2020 19:00,26.728,1940.21,1.64608,27932.57,3396.21,12778.46,44.23,36.32,15344.0,139.55,177.08
16058,08/21/2020 19:15,26.732,1940.00,1.64575,27952.66,3398.47,12793.72,44.23,36.32,15337.8,139.56,177.08
16059,08/21/2020 19:30,26.764,1939.16,1.64556,27940.72,3397.72,12791.71,44.21,36.25,15335.3,139.58,177.08
16060,08/21/2020 19:45,26.652,1937.13,1.64616,27914.10,3395.41,12781.96,44.19,36.29,15334.0,139.60,177.12


In [187]:
dataset.to_csv('silver_daily_2020.csv')